In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fff342ea51d49e23402683c410592072d5487798e3d888065a12dfdfa0cfeaf85aa08c4f9628b97149f851b24e02d0bb1c19821c2f480b284'

In [2]:
from aocd import submit
from aocd.models import Puzzle
day = 11
year = 2025
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [3]:
# test_data = """"""
# data_test = test_data.splitlines()
data_test = puzzle.examples[0].input_data.splitlines()
data_test

['aaa: you hhh',
 'you: bbb ccc',
 'bbb: ddd eee',
 'ccc: ddd eee fff',
 'ddd: ggg',
 'eee: out',
 'fff: out',
 'ggg: out',
 'hhh: ccc fff iii',
 'iii: out']

In [4]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(574,
 ['fxp: udl vii hgb qmy',
  'ixc: cje lra jhu',
  'sgn: apv vbc',
  'wua: knq',
  'fnr: csd awl vvj fyx cvi xhb mvn guw wid vbp uvk krt oil izj zlh gwv',
  'krt: grp cml vxt',
  'hwj: laq jlg',
  'rus: out',
  'oag: wrc gpp',
  'uos: zqg aiu gdx'])

In [207]:
import networkx as nx
import itertools as it
from collections import defaultdict
from functools import cache

In [8]:
def parse(line):
    node, outs = line.split(':')
    outs = outs.strip()
    return node, outs.split(' ') if outs else []

In [22]:
G = nx.DiGraph()
# for line in data_test:
for line in data:
    node, outs = parse(line)
    for out in outs:
        G.add_edge(node, out)

npaths = len(list(nx.all_simple_paths(G, 'you', 'out')))
npaths

585

In [23]:
puzzle.answer_a = npaths

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


# Part 2

In [27]:
test_ = """svr: aaa bbb
aaa: fft
fft: ccc
bbb: tty
tty: ccc
ccc: ddd eee
ddd: hub
hub: fff
eee: dac
dac: fff
fff: ggg hhh
ggg: out
hhh: out"""
data_test_2 = test_.splitlines()

In [210]:
connections = {}
# for line in data_test_2:
for line in data:
    node, outs = parse(line)
    connections[node] = outs
    
@cache
def get_npaths(source, target):
    return source==target or sum(get_npaths(node, target) for node in connections.get(source, []))


In [211]:
res = 1
for s, e in [('svr', 'fft'), ('fft', 'dac'), ('dac', 'out')]:
    res *= get_npaths(s, e)
res

349322478796032

# Old version below

In [ ]:
G = nx.DiGraph()
connections = {}
# for line in data_test_2:
for line in data:
    node, outs = parse(line)
    connections[node] = outs
    for out in outs:
        G.add_edge(node, out)
nx.is_directed_acyclic_graph(G)


True

In [196]:
reversed_topo_sort = list(nx.topological_sort(G))[::-1]
len(reversed_topo_sort)

575

In [198]:
def get_n_simple_paths(source, target, connections):
    stack = [source]
    n = 0
    while stack:
        node = stack.pop()
        for con in connections.get(node, []):
            if con == target:
                n += 1
            else:
                stack.append(con)
    return n

def get_num_paths_dp(G, source, target, connections):
    npaths = defaultdict(int)
    npaths[target] = 1

    def get_npaths(G, node, target, connections, npaths):
        tot = 0
        for con in connections.get(node, []):
            if con in npaths:
                tot += npaths[con]
            else:
                tot += get_n_simple_paths(con, target, connections)
                # tot += len(list(nx.all_simple_paths(G, con, target)))
        return tot

    for node in reversed_topo_sort:
        if node == target:
            npaths[node] = 1
        else:
            npaths[node] = get_npaths(G, node, target, connections, npaths)
        if node == source:
            break

    return npaths[source]

In [199]:
res = 1
for s, e in [('svr', 'fft'), ('fft', 'dac'), ('dac', 'out')]:
    res *= get_num_paths_dp(G, s, e, connections)
res

349322478796032

In [180]:
puzzle.answer_b = res

That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 11! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [ ]:
# 4HbQ

from functools import cache


G = {k[:-1]:v for k,*v in map(str.split, data)} | {'out':[]}

@cache
def count(here, dest):
    return here == dest or sum(count(next, dest) for next in G[here])

# print(count('you', 'out'))

# print(count('svr', 'dac') * count('dac', 'fft') * count('fft', 'out')
#     + count('svr', 'fft') * count('fft', 'dac') * count('dac', 'out'))

In [204]:
@cache
def count(here, dest):
    return here == dest or sum(count(next, dest) for next in connections[here])

In [205]:
count('svr', 'out')

107156221233144865